In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.master('local[*]') \
        .appName('driver-fatigue-average') \
        .getOrCreate()

22/09/14 14:08:31 WARN Utils: Your hostname, ubuntu-linux-20-04-desktop resolves to a loopback address: 127.0.1.1; using 10.211.55.8 instead (on interface eth0)
22/09/14 14:08:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 14:08:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schemaRawDriverFatigue = StructType([
    StructField('bus_id',StringType(),False),
    StructField('status',IntegerType(),True),
    StructField('timestamp',TimestampType(),False),
    StructField('year',IntegerType(),False),
    StructField('month',IntegerType(),False),
    StructField('day',IntegerType(),False),
    StructField('minute',IntegerType(),False),
    StructField('second',IntegerType(),False),
])

In [4]:
parquetDF = spark.read.schema(schemaRawDriverFatigue).parquet('hdfs://localhost:9000/user/parallels/job/driver-fagitue/')

In [5]:
parquetDF = parquetDF.fillna(value=0,subset=['status'])
parquetDF.show(truncate=False)
parquetDF.printSchema()

+--------+------+-----------------------+----+-----+---+------+------+----+
|bus_id  |status|timestamp              |year|month|day|minute|second|hour|
+--------+------+-----------------------+----+-----+---+------+------+----+
|bus_1000|2     |2022-09-13 22:34:00.187|2022|9    |13 |34    |0     |22  |
|bus_1000|0     |2022-09-13 22:34:01.193|2022|9    |13 |34    |1     |22  |
|bus_1000|1     |2022-09-13 22:34:02.2  |2022|9    |13 |34    |2     |22  |
|bus_1000|3     |2022-09-13 22:34:03.208|2022|9    |13 |34    |3     |22  |
|bus_1000|1     |2022-09-13 22:34:04.214|2022|9    |13 |34    |4     |22  |
|bus_1000|0     |2022-09-13 22:34:05.22 |2022|9    |13 |34    |5     |22  |
|bus_1000|3     |2022-09-13 22:34:06.226|2022|9    |13 |34    |6     |22  |
|bus_1000|3     |2022-09-13 22:34:07.233|2022|9    |13 |34    |7     |22  |
|bus_1000|2     |2022-09-13 22:34:08.239|2022|9    |13 |34    |8     |22  |
|bus_1000|3     |2022-09-13 22:34:09.247|2022|9    |13 |34    |9     |22  |
|bus_1000|2 


[Stage 0:>                                                          (0 + 1) / 1]



In [12]:
RawDriverFatiguePerSecond = parquetDF.select('bus_id','status','year','month','day','hour','minute','second')
RawDriverFatiguePerMinute = RawDriverFatiguePerSecond.select('bus_id','status','year','month','day','hour','minute')
RawDriverFatiguePerHour = RawDriverFatiguePerMinute.select('bus_id','status','year','month','day','hour')
RawDriverFatiguePerDay = RawDriverFatiguePerHour.select('bus_id','status','year','month','day')
RawDriverFatiguePerMonth = RawDriverFatiguePerDay.select('bus_id','status','year','month')
RawDriverFatiguePerYear = RawDriverFatiguePerMonth.select('bus_id','status','year')
RawDriverFatiguePerSecond.show(truncate=False)

+--------+------+----+-----+---+----+------+------+
|bus_id  |status|year|month|day|hour|minute|second|
+--------+------+----+-----+---+----+------+------+
|bus_1000|2     |2022|9    |13 |22  |34    |0     |
|bus_1000|0     |2022|9    |13 |22  |34    |1     |
|bus_1000|1     |2022|9    |13 |22  |34    |2     |
|bus_1000|3     |2022|9    |13 |22  |34    |3     |
|bus_1000|1     |2022|9    |13 |22  |34    |4     |
|bus_1000|0     |2022|9    |13 |22  |34    |5     |
|bus_1000|3     |2022|9    |13 |22  |34    |6     |
|bus_1000|3     |2022|9    |13 |22  |34    |7     |
|bus_1000|2     |2022|9    |13 |22  |34    |8     |
|bus_1000|3     |2022|9    |13 |22  |34    |9     |
|bus_1000|2     |2022|9    |13 |22  |34    |10    |
|bus_1000|1     |2022|9    |13 |22  |34    |11    |
|bus_1000|1     |2022|9    |13 |22  |34    |12    |
|bus_1000|2     |2022|9    |13 |22  |34    |13    |
|bus_1000|1     |2022|9    |13 |22  |34    |14    |
|bus_1000|1     |2022|9    |13 |22  |34    |15    |
|bus_1000|0 

In [7]:
# parquetDF = parquetDF.withColumn('event_window', window(col('timestamp'),'1 minutes'))
# parquetDF.show(truncate=False)

In [8]:
groupByMin = parquetDF.groupBy('bus_id','year','month','day','hour','minute').agg(avg('status'),count('status')).orderBy(col('bus_id').asc(),col('year').desc(),col('month').desc(),col('day').desc(),col('hour').desc(),col('minute').desc())
groupByMin.show(truncate=False)

+--------+----+-----+---+----+------+------------------+-------------+
|bus_id  |year|month|day|hour|minute|avg(status)       |count(status)|
+--------+----+-----+---+----+------+------------------+-------------+
|bus_1000|2022|9    |13 |23  |7     |1.9166666666666667|12           |
|bus_1000|2022|9    |13 |23  |6     |1.396551724137931 |58           |
|bus_1000|2022|9    |13 |23  |5     |1.3859649122807018|57           |
|bus_1000|2022|9    |13 |23  |4     |0.0               |1            |
|bus_1000|2022|9    |13 |22  |45    |1.3333333333333333|24           |
|bus_1000|2022|9    |13 |22  |44    |1.576271186440678 |59           |
|bus_1000|2022|9    |13 |22  |43    |1.5               |60           |
|bus_1000|2022|9    |13 |22  |42    |1.4915254237288136|59           |
|bus_1000|2022|9    |13 |22  |41    |1.4166666666666667|60           |
|bus_1000|2022|9    |13 |22  |40    |1.6271186440677967|59           |
|bus_1000|2022|9    |13 |22  |39    |1.55              |60           |
|bus_1